In [5]:
import json

In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter

# Функція для генерації випадкових даних
def generate_data(n):
    age = np.random.choice(['<30', '30-40', '>40'], n)
    job = np.random.choice(['Немає', 'Є'], n)
    credit_history = np.random.choice(['Погана', 'Нормальна', 'Хороша'], n)
    income = np.random.choice(['Низькі', 'Середні', 'Високі'], n)
    loan = []

    for i in range(n):
        if age[i] == '<30':
            if credit_history[i] == 'Хороша' or (job[i] == 'Є' and income[i] == 'Високі'):
                loan.append('Так')
            else:
                loan.append('Ні')
        elif age[i] == '30-40':
            if credit_history[i] != 'Погана' or income[i] == 'Високі':
                loan.append('Так')
            else:
                loan.append('Ні')
        else:  # age[i] == '>40'
            if job[i] == 'Є' and credit_history[i] != 'Погана':
                loan.append('Так')
            else:
                loan.append('Ні')

    return pd.DataFrame({
        'Вік': age,
        'Робота': job,
        'Кредитна історія': credit_history,
        'Доходи': income,
        'Видача кредиту': loan
    })

# Генеруємо вибірку даних на 100 записів
df = generate_data(100)

In [2]:
df

,Вік,Робота,Кредитна історія,Доходи,Видача кредиту
0,30-40,Немає,Хороша,Середні,Так
1,<30,Є,Нормальна,Високі,Так
2,>40,Є,Нормальна,Середні,Так
3,30-40,Немає,Хороша,Низькі,Так
4,<30,Є,Нормальна,Середні,Ні
...,...,...,...,...,...
95,30-40,Немає,Хороша,Високі,Так
96,30-40,Є,Хороша,Низькі,Так
97,<30,Є,Погана,Середні,Ні
98,30-40,Є,Погана,Низькі,Ні


In [3]:
df.head(5)

,Вік,Робота,Кредитна історія,Доходи,Видача кредиту
0,30-40,Немає,Хороша,Середні,Так
1,<30,Є,Нормальна,Високі,Так
2,>40,Є,Нормальна,Середні,Так
3,30-40,Немає,Хороша,Низькі,Так
4,<30,Є,Нормальна,Середні,Ні


In [4]:
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum([(-counts[i]/np.sum(counts)) * np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

# Функція для обчислення інформаційного виграшу
def InfoGain(data, split_attribute_name, target_name="Видача кредиту"):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = np.sum([(counts[i]/np.sum(counts)) * entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - weighted_entropy
    return Information_Gain

# Функція для створення дерева рішень
def ID3(data, originaldata, features, target_attribute_name="Видача кредиту", parent_node_class=None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]
    elif len(features) == 0:
        return parent_node_class
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature: {}}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data, originaldata, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
        return tree

# Створюємо дерево рішень
features = list(df.columns)
features.remove("Видача кредиту")
tree = ID3(df, df, features)
print(tree)

{'Кредитна історія': {'Нормальна': {'Вік': {'30-40': 'Так', '<30': {'Доходи': {'Високі': 'Так', 'Низькі': 'Ні', 'Середні': 'Ні'}}, '>40': {'Робота': {'Є': 'Так', 'Немає': 'Ні'}}}}, 'Погана': {'Доходи': {'Високі': {'Вік': {'30-40': 'Так', '<30': {'Робота': {'Є': 'Так', 'Немає': 'Ні'}}, '>40': 'Ні'}}, 'Низькі': 'Ні', 'Середні': 'Ні'}}, 'Хороша': {'Вік': {'30-40': 'Так', '<30': 'Так', '>40': {'Робота': {'Є': 'Так', 'Немає': 'Ні'}}}}}}


In [9]:
data_json = df.to_json(orient='records', force_ascii=False, indent=4)

with open('data.json', 'w', encoding='utf-8') as file:
    file.write(data_json)

print("Data has been written to 'data.json'")

Data has been written to 'data.json'
